In [ ]:
import argparse
import os
import json
import tqdm
import math
import numpy as np
import random
import time
import torch
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from PIL import Image
import pprint
import pandas as pd

np.random.seed(42)
random.seed(42)

from model import Seq2SeqAutoEncoderModel
from evaluation import decode_image_from_seq, visualize_segments, tsne_visualize, get_knn_similarity, get_datasets, knn_evaluation, reconstruction_evaluation

In [ ]:

os.environ['CUDA_VISIBLE_DEVICES'] = '4'
# model_dir = '/home/dchenbs/workspace/Seq2Seq-AutoEncoder/runs/Nov22_19-50-39_host19-SA1B-[330MB-16queries-2304]-[lr1e-05-bs4x4step-8gpu]/checkpoints/checkpoint_ep0_step600k'
model_dir = '/home/dchenbs/workspace/Seq2Seq-AutoEncoder/runs/Nov14_17-31-06_host19-SA1B-[327MB-16queries-1024]-[lr1e-05-bs16x1step-8gpu]/checkpoints/checkpoint_ep0_step1000k'

print(f'Loading model from {model_dir}')

model = Seq2SeqAutoEncoderModel.from_pretrained(model_dir).cuda().eval()
datasets = get_datasets(model)
datasets = datasets[:1]

num_steps = 100
batch_size = 100
k=5
knn_evaluation_results = knn_evaluation(
    model, 
    datasets, 
    num_steps=num_steps, 
    batch_size=batch_size, 
    k=k, 
    visualize=True
    )
file_name = f'knn_evaluation_k={k}_{num_steps*batch_size}samples'
json.dump(knn_evaluation_results, open(os.path.join(model_dir, f'{file_name}.json'), 'w'), indent=4)
pd.json_normalize(knn_evaluation_results, sep='-').to_csv(os.path.join(model_dir, f'{file_name}.csv'), index=False)
print(f'>>> Saved KNN Evluation Results to {model_dir}/{file_name}.json/csv')

num_steps = 10
batch_size = 10
reconstruction_evaluation_results = reconstruction_evaluation(
    model, 
    datasets, 
    num_steps=num_steps, 
    batch_size=batch_size, 
    visualize=True
    )
file_name = f'reconstruction_evaluation_{num_steps*batch_size}samples'
json.dump(reconstruction_evaluation_results, open(os.path.join(model_dir, f'{file_name}.json'), 'w'), indent=4)
pd.json_normalize(reconstruction_evaluation_results, sep='-').to_csv(os.path.join(model_dir, f'{file_name}.csv'), index=False)
print(f'>>> Saved Reconstruction Evaluation Results to {model_dir}/{file_name}.json/csv')